In [1]:
from floridaman import data_cleaning
import pandas as pd
import numpy as np

In [2]:
# Load data
raw_data = data_cleaning.load('null_transformed')

In [3]:
# Generate sample dataset with the following parameters
# Source dataframe: raw_data
# Column drop threshold: 40%
# Column impute threshold: 20%
# n_neighbors for KNN imputation
this_data = data_cleaning.generate_candidate_dataset(raw_data, .40, .35, 5)

In [4]:
# Generate X and y dataframes for model input
X = np.array(this_data[data_cleaning.features(this_data)])
y = np.array(this_data['FAILURETYPE'])

In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

max_score = 0

print("Training progress: ", end = '')
for COL_DROP in range(1,100,5):
    print('*', end = '')
    for IMPUTE in range(1, COL_DROP, 1):
        for NEIGHBOR in range(1, 20, 2):
            COL_DROP = COL_DROP/100
            IMPUTE = IMPUTE/100
            
            knn = KNeighborsClassifier(n_neighbors=NEIGHBOR)
            this_score = cross_val_score(knn, X, y, cv=8, scoring='roc_auc_ovo')
            this_score = this_score.mean()
            
            if (this_score > max_score):
                best_mode = [COL_DROP, IMPUTE, NEIGHBOR, this_score]

In [6]:
this_sample = data_cleaning.balance(this_data)

In [7]:
this_sample['FAILURETYPE'].value_counts()

2    161
1    161
0    161
Name: FAILURETYPE, dtype: int64

In [10]:
this_data

,roduid,UWI,lifetime_start,lifetime_end,FAILURETYPE,H2S_CONCENTRATION,PrimarySetpoint,SecondarySetpoint,StrokeLength,Fillage,...,rod_make,rod_apigrade,ROUTE,overall_max_sideload,shallow_max_sideload,max_unguided_sideload,DESANDDEGAS_TYP,NIPPLE_SET_DEPTH,pump_bore,lifetime
0,440,0,2019-07-16,2020-02-13,2,0.0,80.0,65.0,165.878957,98.000000,...,1,3,903,174.270,174.270,174.270,1,8893.9,2.00,212
3,787,1,2017-10-18,2018-05-15,2,0.0,70.0,65.0,165.761084,99.528572,...,1,3,905,356.266,353.938,188.798,1,11505.2,1.25,209
4,771,1,2018-05-18,2018-07-18,1,0.0,70.0,65.0,168.259995,98.699997,...,1,3,905,367.298,367.298,290.388,1,11387.1,1.25,61
5,64,2,2017-07-02,2020-01-31,1,0.0,76.0,60.0,144.000000,68.096428,...,2,0,883,153.430,153.430,105.290,1,9847.1,1.50,943
6,109,2,2016-02-17,2017-06-29,1,0.0,76.0,65.6,144.000000,90.614708,...,2,0,883,218.330,218.330,90.940,1,9847.8,1.50,498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1712,125,480,2017-04-04,2018-01-03,2,0.0,85.0,60.0,169.429774,85.362503,...,1,3,873,298.080,298.080,176.860,2,8975.0,1.75,274
1717,411,481,2018-08-28,2020-03-21,2,0.0,80.0,65.0,144.039713,27.100000,...,1,3,873,184.966,184.966,143.214,2,9714.3,1.50,571
1718,594,482,2016-11-05,2017-08-28,2,0.0,75.0,60.0,144.039858,71.629166,...,2,3,874,142.454,140.878,142.454,0,9598.3,1.75,296
1719,68,482,2015-01-23,2016-11-03,2,0.0,75.0,60.0,144.000000,68.190002,...,2,3,874,113.210,113.210,113.210,0,9676.4,1.75,650
